# Model framework

"Approximately 70% of problems in Data Science are classification problems." (Bron: DataCamp)

Classification models zijn modellen die worden ingezet om datapunten te classificeren. Voorbeelden van cases waarbij classification wordt ingezet zijn:
- de gezondheidszorg: gegeven de levensstijl en symptonen, wordt iemand ziek of niet?
- de verzekeringswereld: gegeven deze claims en onderbouwing, fraudeert iemand of niet?
- de bank: kan een klant zijn lening terugbetalen of niet? *zie illustratie*
- marketing: zien we een klant nog terug? (churn)

Churn is wellicht het bekendste voorbeeld in ons vakgebied, maar er zijn nog genoeg andere voorbeelden te bedenken waarin je als marketing analist informatie wil krijgen over wie een bepaalde actie gaat uitvoeren/zal ondergaan én waarom.

<img src="https://cdn-images-1.medium.com/max/1600/0*QUP2zFOd-pzmM3cW.">

Dit notebook is bedoeld om een een classification model te bouwen volgens de Eneco MI structuur. Deze structuur is grotendeels gebaseerd op de industrie standaard van het data science proces (<a href="https://en.wikipedia.org/wiki/Cross-industry_standard_process_for_data_mining">CRISP DM</a>). Belangrijk om te beseffen is dat het maken van een (classificatie) model een iteratief proces is waarbij je niet in een rechte lijn van a naar z gaat, maar vaak terugkeert naar vorige stappen om aanpassingen te maken op basis van nieuwe inzichten.

<img src="https://miro.medium.com/max/1200/1*2NajmK58hJf8lJQm25iXWw.png" width=400>

Deze Eneco MI structuur bestaat uit 6 stappen die later in het notebook nader zullen worden toegelicht.
- Stap 1: data import
- Stap 2: data exploration
- Stap 3: data preparation
    - 3a: cleaning
    - 3b: feature engineering
    - 3c: feature scaling
- Stap 4: feature selection by modeling and / or business sense 
- Stap 5: modeling & evaluation
- Step 6: uitscoren

Na het doorlopen van dit notebook heb jij een classification model gebouwd! Aangezien niet iedereen even ervaren is in het bouwen van modellen hebben we per stap suggesties gegeven om je kennis weer op te frissen of voor verdere verdieping.
Mocht je bepaalde functies niet begrijpen of wil je meer weten over een package raadpleeg dan eerst de online documentatie of roep de help functie aan in Python:

```python
print(help(*naam van de functie*))
```

Heb je naar aanleiding van dit script nog vragen over werken met Python? Schroom dan niet om contact op te nemen met DDM (sander.knol@eneco.com) of het Data Science Customers & Operations team (datascience@eneco.com). 

Run allereerst onderstaande twee cellen om dit notebook werkzaam te krijgen met de meeste recente functies.

In [ ]:
from src.features.prep_data import copy_df, describe_df
from .. import return_data_dwh
from .. import ..
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# dit is niet nodig tenzij ze zelf nieuwe functies gaan schrijven, mogelijk goed om uit te leggen?
%load_ext autoreload
%autoreload 2

## Stap 1: data import

In deze eerste stap wordt een dataset geïmporteerd waarop het model wordt gebouwd. DDM heeft een SQL script klaargezet dat dient als uitgangspunt voor de trainingsset die gebruikt wordt voor modelbouw. In het script staat uitleg over de aanpassingen die je moet maken om de set te laten aansluiten bij jouw vraagstuk. Het script vind je [hier](https://eneco.sharepoint.com/sites/econts02x/85/Team/Forms/AllItems.aspx?RootFolder=%2Fsites%2Feconts02x%2F85%2FTeam%2FENECO%5FP%2FPROJECTEN%2FVoorspelmodellen%20bouwen&FolderCTID=0x012000476CE4D54C7C7246A549975C9A513BA4&View=%7B7534D727%2D5B74%2D40D7%2DA354%2D9F0206F8BDDE%7D).

Als je de resultaten van het script wegschrijft in een DWH tabel, dan kun je deze tabel vervolgens als Pandas dataframe importeren naar dit notebook. Dit kun je doen met behulp van de functie hieronder. Pas hiervoor in de regel waarin je de functie aanroept je personeelsnummer aan, de query (select * from tabelnaam), de locatie waar je SQL-wachtwoord staat en de omgeving waarop je wil inloggen (OLAB = 1, LAB = 2, zonder aanduiding = 3). Let op, met passloc wordt bedoeld een bestandje dat je lokaal op je pc hebt opgeslagen waarin je wachtwoord vermeld staat. Door daarnaar te verwijzen hoef je je wachtwoord nooit hardcoded in Python code te schrijven wat voorkomt dat je je wachtwoord per ongeluk deelt met collega's. 

In [ ]:
df = return_data_dwh(pnumber, query, passloc, usernumber)

**Belangrijk**:
Zodra de data goed is ingeladen is het belangrijk om een deel van de dataset opzij te zetten om mee te testen hoe goed je finale model presteert. We noemen dit de test set (of holdout set) en deze stap vindt dus nog vóór de data exploratie fase plaats. In onderstaand voorbeeld zetten we random 20% van de totale set opzij. Dit percentage kan naar gelieve worden aangepast, maar moet hoog genoeg zijn om een significante test te bewerkstelligen. Door een waarde in te vullen bij random_state, wordt bewaard welke variabelen random zijn gekozen (in feite is er dan ook sprake van pseudo-randomization). Het is belangrijk dat je deze holdout set verder tijdens het modelbouw proces ongemoeid laat. Wanneer je bijna alle stappen doorlopen hebt en een goed werkend model hebt getraind dan geeft de evaluatie op deze holdout set het beste beeld van de te verwachten performance op compleet nieuwe data.

In [ ]:
train_set, test_set = train_test_split(df, test_size=0.20, random_state=1337)

## Stap 2: data exploratie

De data exploratie fase is een eerste kennismaking met de dataset. Het doel is om vertrouwd te raken met de dataset en om ideeën uit te werken voor de volgende fases. Belangrijke vragen in deze fase:
- is de data goed ingeladen?
- zijn er missing values?
- welke variabelen hebben outliers en hoe zijn die ontstaan?
- hoe zit het met de correlatie tussen de variabelen?
- zijn er variabelen die weinig tot niets toevoegen (near-zero variance)?
- ..

Om inzage te krijgen in deze belangrijke vragen hebben we reeds functies ontwikkeld die je daarbij kunnen helpen: 

<tr>
    <td>
    <img src="https://www.researchgate.net/profile/Sil_Aarts/publication/323072965/figure/fig1/AS:758041565724674@1557742601380/Figuur-1-Histogram-voor-de-verdeling-van-leeftijd-van-626-deelnemers-Gemiddelde-66-94_Q320.jpg" style="width: 400px;"> 
    </td>
    <td>   
<img src="https://machinelearningmind.files.wordpress.com/2019/10/screenshot-2019-10-19-at-2.00.35-pm.png?w=431" style="width: 400px;">
    </td>
</tr>

Onderstaande functie geeft een algemene beschrijvende statistiek van de dataset, zoals #rijen, #kolommen, verdeling van type variabelen, etc.

In [ ]:
describe_df(df, dependent_variable='Y_VAR')

Als twee of meer variabelen in hoge mate hetzelfde zeggen, dan kan hetgeen wat ze zeggen te zwaar worden meegenomen in je model. Er is dan sprake van multicollineariteit. Onderstaande functie geeft weer welke combinaties van variabele grotendeels hetzelfde gedrag vertonen en geeft een suggestie welke van deze twee variabelen dan het beste weggelaten kan worden. Het cutoff punt kun je zelf bepalen, maar staat default op 0.95.

In [ ]:
multicollinearity(df, cut_off)

In [ ]:
** Provides the 5 most prevalent values per variable and the absolute difference between the top 2 values.

    This insight can be used to determine which variables carry very little information or are interesting for
    further feature engineering.
abs : absolute difference between the two most prevalent values

In [ ]:
freq_insight(df)

Met de functie hier beneden worden de categorische variabelen uit de trainingsset in een grafiek gevisualiseerd met op de x-as de categorie en op de y-as het gemiddelde van de dependent variable voor alle klanten die in deze categorie vallen. Op die manier krijg je een eerste inzicht in of de variabele mogelijk effect heeft op de dependent variable. 

Als je de trainingsset zoals eerder in dit notebook geadviseerd hebt gebruikt, zijn de categorische variabelen allemaal afkomst van infobase. In die set wordt zowel een 5-punts als een 7-punts schaal gebruikt. In de functie zijn alle mogelijke categorieën op de x-as gezet. Deze functie is dus niet geschikt om te gebruiken voor een trainingsset met zeer veel categorische variabelen die allemaal andere categorieën bevatten.

In [ ]:
cat_visu(df)

Ook onderstaande functie maakt inzichtelijk wat de gemiddelde waarde van de y-variabele is bij elke waarde van de x-variabele, maar doet dit voor binaire variabelen. Wederom geeft dit inzicht in de mate waarin een independent variable informatie toevoegt ten aanzien van de dependent variable.

In [ ]:
odds_ind(df)

Onderstaande functie laat de verdeling van numerieke variabelen zien. Op die manier kan worden afgeleid of er sprake is van een normaalverdeling, scewness, een gebrek aan variantie of bijvoorbeeld een duidelijke tweedeling in de data.

In [ ]:
num_visu(df)

Principal component analysis is een analysemethode die wordt ingezet om het aantal variabelen wat je gebruikt om je gebruikt in je model te reduceren. Simpel gezegd is het een methode die de bestaande variabelen in de set vervangt door een nieuwe, kleinere, set aan variabelen. Deze kleinere set aan variabelen is eigenlijk een set van dimenties die zijn opgesteld door een combinatie en transformatie van de originele data uit de set. Hoe principal component analysis (PCA) precies werkt wordt goed uitlegd in [dit artikel](https://georgemdallas.wordpress.com/2013/10/30/principal-component-analysis-4-dummies-eigenvectors-eigenvalues-and-dimension-reduction/).

Het resultaat van onderstaande functie geeft weer hoeveel variantie van de y-variable kan worden bepaald met welke hoeveelheid dimenties/prinicpal axis.

In [ ]:
pca_visu(df)

## Stap 3: data preperation

Nu je meer inzage hebt in je dataset gaan we de meest tijdrovende fase in: data preparation. In deze fase wordt grotendeels de performance van het model bepaald dus onderschat dit niet. Data preparation betaat uit 3 onderdelen:
- cleaning
- feature engineering
- feature selection

Ieder onderdeel zal afzonderlijk worden toegelicht, hou er rekening mee dat je in de fase vaak terug zal keren naar een vorig onderdeel, dit is normaal en hoort bij het iteratieve karakter van het data science proces.

### Stap 3a: cleaning

"60 to 80 percent of the total time is spent on cleaning the data before you can make any meaningful sense of it." (Bron: Towards Data Science)

Om waardevollen inzichten uit je data te kunnen extraheren, moet je data allereerst geschoond worden om ruis weg te nemen en effecten bloot te kunnen leggen. We spreken van schone data als de data hoogt scoort op, validiteit, accuraatheid, compleetheid, consistency en uniformiteit. Onderstaand overzicht laat zien waar je aan moet denken bij data cleaning.

<img src="https://3jd8gl2iires146kaw2hgqy9-wpengine.netdna-ssl.com/wp-content/uploads/2017/01/Data-cleaning-checklist-v2-01.png" width=600>

Een onderdeel van data cleaning is dus het omgaan met missing values. Hoe je daar het beste mee kan omgaan, hangt af van de volgende aantal factoren:
- Wat is het type missing? MCAR/MAR/MNAR? Lees hier meer over op https://towardsdatascience.com/practical-strategies-to-handle-missing-values-626f9c43870b?gi=dcd45101e7ac
- Hoeveel procent van de cellen in een kolom is leeg?

Door onderstaande cell te runnen worden de variabelen met een groot aantal missende waarden verwijderd. Hoe groot dit aantal is kan worden bepaald door een threshold te zetten. Alle variabelen waarvan het aandeel missings groter is dan de threshold zal worden verwijderd uit de trainingsset. Welke variabelen dit zijn is terug te zien in het resultaat van de vorige stap, waar perc_missings per column wordt weergegeven. Gebruik deze functie voor kolommen met veel missings en/of MCAR.

In [ ]:
drop_missings(df, threshold=0.2)

In plaats van het verwijderen van kolommen met lege cellen kan ook worden gekozen om deze lege cellen op te vullen. Met het runnen van onderstaande functie worden deze lege cellen opgevuld met de mediaan van de kolom. Gebruik deze functie voor kolommen met weinig missings en/of MCAR/MAR.

In [ ]:
median_impute(df)

Als een count variabele missende waarden bevat, is de kans de groot dat dit betekent dat de telling van de waarde 0 is. In dan geval kun je bij de lege cellen een 0 invullen met onderstaande functie.

In [ ]:
fill_na_zero(df)

Als een missing niet volledig random is en er dus een reden is voor de missing, kan deze reden meer zeggen over de kolom dan de variabele zelf. In dat geval kan het slim zijn die informatie mee te nemen in een nieuwe variabele.

In [ ]:
drop_high_correlated(df)

Op het moment dat een variabele weinig tot geen variantie bevat (dat wil zeggen dat de waarden nagenoeg allemaal gelijk zijn), dan geeft die variabele geen informatie over de dependent variable en is deze overbodig. Daarom kun je een variabele met near zero vero uit de dataset verwijderen met onderstaande functie.

In [ ]:
nzv(df)

#### Overige aanpassingen

Tenslotte zijn er ook nog een aantal overige aanpassingen die wellicht gewenst zijn voor het bouwen van het model dat je wil. Zo kan het een logische veronderstelling zijn dat klanten van verschillende labels zich anders gedragen en dat je daarom een model wil maken voor een specifiek label. Daarnaast zitten er misschien variabelen in de set waarvan je op basis van common sense vooraf al kan zeggen dat zij niet van invloed kunnen zijn op je Y-variabelen of wellicht heb je een andere reden om ze uit te sluiten. Tenslotte is het raadzaam de titels van je variabelen alleemaal lowercase (of uppercase) te maken zodat je niet iedere keer terug hoeft te kijken hoe de variabele wordt geschreven.

In [ ]:
select_owner(df, owner)
standaard_drops(df, list_of_variables)
col_to_lower(df)

### Stap 3b: feature engineering

#### Alter data types

Python Pandas werkt met een aantal datatypes, welke veelal zijn afgeleid van NumPy. De belangrijkste en meest voorkomende types zijn opgenomen in onderstaand overzicht.

<img src="https://pbpython.com/images/pandas_dtypes.png">

Hieronder worden een aantal suggesties gegeven om het dataypes van de variabelen in je set aan te passen. Run eerst onderstaand blok om te ontdekken welk variabele in je dataset welke datatype heeft.

In [ ]:
print(df.dtypes())

Veel modellen kunnen niet omgaan met categorische variabelen. Daarom is het verstandig deze te hercoderen naar dummy-variabelen. Dat doet onderstaande functie. Let op: je bent misschien gewend dat het aantal dummies gelijk is aan het aantal categorieën - 1. Bij deze functie is dat niet het geval, hier krijg je evenveel indicator variabelen als het aantal categorieën waar de originele variabele uit bestaat.

In [ ]:
to_dummies(df, list_of_variables)

#### Dummy encoding 

Machine learning algoritmes kunnen over het algemeen *alleen* omgaan met numerieke waarden. Voor het TCS model wouden we echter gebruik maken van de energielabel, en die variabele is categorisch (object, zie datatypes). Een manier om hiermee om te gaan is het maken van dummy variabelen, en deze functie zit standaard in de Pandas package.

Het idee is om van ieder label een numerieke indicator te maken, zodat alle informatie beschikbaar blijft maar dan numeriek.

In [1]:
import pandas as pd
df = pd.DataFrame({'label': ['A', 'B', 'C', 'D', 'E']})
print(df)
print(df['label'].dtype)

  label
0     A
1     B
2     C
3     D
4     E
object


In [2]:
print(pd.get_dummies(df))
print(pd.get_dummies(df).dtypes)

   label_A  label_B  label_C  label_D  label_E
0        1        0        0        0        0
1        0        1        0        0        0
2        0        0        1        0        0
3        0        0        0        1        0
4        0        0        0        0        1
label_A    uint8
label_B    uint8
label_C    uint8
label_D    uint8
label_E    uint8
dtype: object


Misschien staat er waarden in je dataset die wel numeriek zijn, maar die Pandas niet als zodanig herkent. Dat kan problemen veroorzaken: Python snapt niet 1 + 2 = 3 niet als 1 wordt gezien als een stukje tekst in plaats een cijfer. Met onderstaande functie kun je een datatype veranderen naar numeriek. Doe dit door je dataframe aan te roepen en een lijst van de variabelen die je numeriek wil maken.

In [ ]:
to_numeric(df, list_of_variables)

Een ander datatype issue gaat over numerieke ordinale variabelen en ratio variabelen. Beide zijn integers of floats, maar bij een binaire indicator geeft een 1 een categorie aan en geen aantal. Daarom wil je dat indicatoren ook als dusdanig worden behandeld en niet worden gezien als een frequency variabelen. Onderstaand blok lost dat voor je op. Let er wel op dat je deze functie altijd gebruikt nadat je de to_dummies functie hebt gebruikt, omdat je met die functie indicator variabelen aanmaakt.

In [ ]:
freq_to_ind(df, list_of_variables)

#### Transform data

Veel statistische technieken vereisen dat er aan bepaalde aannames wordt voldaan, zoals normaalverdeling, multicollinearity of homoscedasticity. [Dit artikel](https://towardsdatascience.com/all-the-annoying-assumptions-31b55df246c3) legt uit hoe het ook alweer zat met die aannames. 

Wanneer je data niet voldoet aan de aannames van het model, zijn er een aanal datatransformaties mogelijk.

Eén daarvan is het vervangen van de waarden door de wortel van deze waaarden. Deze transformatie is bijzonder geschikt voor: 
1. correctie voor de schending van de aanname van homogeniteit van variantie
2. het corrigeren van data met een skew naar rechts naar (meer) normaal verdeelde data
3. het algemeen verbeteren van lineaire fit

Onderstaande functie voert die transformatie uit.

In [ ]:
sqrt_transformer(df, list_of_variables)

Bij een log-transformatie  wordt het natuurlijk logaritme van de originele data genomen. Deze transformatie methode wordt afgeraden voor datasets/variabelen met negatieve waardes. Deze transformatie is handig voor: 
1. het corrigeren voor de schending van de aanname van homogeniteit van variantie
2. het corrigeren van een sterke skew
3. het algemeen verbeteren van lineaire fit.

Je kunt een log-transformatie aanroepen voor een lijst van variabelen met de functie hier beneden.

In [ ]:
log_transform(df, list_of_variables)

#### Feature engineering examples

"Coming up with features is difficult, time-consuming, requires expert knowledge. "Applied machine learning" is basically feature engineering."<br>
-- Andrew NG, *Machine Learning and AI via Brain simulations*

Hier boven zijn een aantal voorbeelden gegeven van standaard datatransformaties die je kunt toepassen. Feature engineering is echter vooral een stuk waarin je zelf veel creativiteit en business kennis kwijt kunt! Hoe transformeer je je gegevens dusdanig dat het model wat je later gaat bouwen zo goed mogelijk in staat is de juiste informatie uit je data te halen? In feite is het antwoord op deze vraag volledig afhankelijk van je vraagstuk dat je wil oplossen met je model. Misschien hebben eerdere analyses al aangetoond dat een interactie tussen twee bepaalden variabelen een effect heeft op je y-variabele, weet je dat uitschieters veroorzaakt worden door fouten in de data of ben je benieuwd naar het effect van een bepaalde factor die niet direct is terug te herleiden naar één variabele uit je set.

Om je een beetje een richting te geven in te transformaties waar je aan zou kunnen denken, geven we hieronder alvast wat voorbeelden. Maar let op: deze voorbeelden zijn niet voor iedere toepassing bruikbaar en bovenal zijn er nog veel meer transformatie denkbaar! Voor hulp of advies kun je altijd bij DDM terecht.

Machine learning betekent dat we een machine gaan trainen om iets te leren maar besef goed dat deze machine an sich niet intelligent is. Om de machine zo goed mogelijk te laten leren moet je het de machine zo makkelijk mogelijk maken om te datgene te leren waar jij interesse in hebt. Om dit concept duidelijk te maken volgen enkele voorbeelden uit het Toon cross-sell (TCS) model.

#### Expert kennis: duurzaamheid

Zoals beschreven in de quote gaat feature engineering over het inzetten van expert kennis om het maximale uit het ML algoritme te halen. Een voorbeeld uit het Toon cross-sell model is een gecombineerde 'duurzaamheid' variabele. Deze variabele bevat de som van de volgende 4 indicator variables (1/0):
- wind energie uit Nederland
- wind energie uit Europa
- zon
- ecogas

Het idee om deze variabelen te combineren ontstond doordat:
- we het idee hadden dat Toon voor sommige klanten mogelijk een duurzame keuze is, die zou kunnen correleren met andere bewuste keuzes
- iedere losse variabele weinig informatie bevat (veel 0-en, 'sparse') maar gecombineerd meer informatie bevat

De winst is dat we de informatie van 4 variabelen kunnen vatten in 1 en iedere losse variabele geen associatie met de dependent variable vertoonde maar de som van de variabelen wel:

<img src="img/toon_duurzaam.PNG">

#### Expert kennis: woonwaarde

Om het Toon cross-sell model te verbeteren hebben we ook gekeken naar externe variabelen, zoals de waarde van de woning. Een van de variabelen die we onderzochten was woning_waarde_cat. Dit is een (numerieke) categorische variabele met 7 levels:
- 1: geen koopwoning
- 2: < 75.000 euro
- ..
- 6: 350.000 - 500.000 euro
- 7: > 500.000 euro

Ook voor deze variabele onderzochten we de associatie met Toon cross-sell per level:

<img src="img/woning_waarde.PNG">

Levels 3, 4, en 5 laten een (licht) verhoogde associatie zien met Toon cross-sell. We zien hier dus wel een effect van woningwaarde op Toon cross-sell, maar dit verband is niet lineair en er is geen éénduidig cutoff punt vast te stellen. Belangrijk om te beseffen is dat lang niet alle ML algoritmes deze associatie goed kunnen modelleren. Een simpel voorbeeld is een decision tree, bij welke level maak je de cutoff om Toon cross-sell zo goed mogelijk te voorspellen? Om het ML algoritme te helpen hebben we een extra indicator variable toegevoegd:
```python
df["nf_woning_waarde_ind"] = np.where(df["woning_waarde_cat"].isin([3, 4, 5]), 1, 0)
```

### Stap 3c: feature scaling

Het 'schalen' van data is een belangrijk onderdeel in het trainen van een ML model, in sommige gevallen maakt het zelfs een verschil tussen een slecht of goed model. Maar wat is het precies en waarom maakt het uit?

Het schalen van de data betekent dat je distributie van de ruwe data verandert naar een schaal met een gemiddelde van 0 en een standaard deviatie van 1. Het klinkt handig dat features dezelfde schaal hebben maar waarom is het belangrijk?:
- sommige ML algoritmes maken gebruik van afstand
- sommige pre-processing technieken worden gestuurd door de absolute variantie
- sommige ML algoritmes zoeken iteratief naar de beste oplossing

K-means en K-Nearest-Neighbours (KNN) zijn 2 bekende ML algoritmes die werken op basis van de afstand tussen
verschillende datapunten. Stel je de volgende situatie voor:
- x1: aantal kamers [1-6]
- x2: originele vraagprijs [150.000 - 1.000.000]

Wanneer je met deze data zou gaan werken zou *alleen* de originele vraagprijs invloed hebben aangezien de ruwe afstanden met betrekking tot het aantal kamers compleet worden ondergesneeuwd.

Principal components analysis (PCA) is een pre-processing techniek om dimensies te reduceren en zoekt naar features met de hoogste absolute variantie. Zonder te schalen zal wederom alleen de orginele vraagprijs mee worden genomen aangezien die verschillen vele malen groter zijn. Hieronder een voorbeeld, het is niet erg als je de code nog niet helemaal begrijpt.

In [38]:
import numpy as np
from sklearn.preprocessing import StandardScaler

test_data = pd.DataFrame({'aantal_kamers': np.random.randint(low=1, high=6, size=1000),
                          'vraagprijs': np.random.randint(low=150000, high=1000000, size=1000)})

print(test_data.head())

   aantal_kamers  vraagprijs
0              4      626329
1              3      333703
2              5      741123
3              5      180405
4              2      552493


In [32]:
test_data.std() # standaard deviatie, sqrt(variance)

aantal_kamers         1.419326
vraagprijs       246427.718265
dtype: float64

In [39]:
scaler = StandardScaler()
test_data_scaled = scaler.fit_transform(test_data)
test_data_scaled = pd.DataFrame(test_data_scaled, columns=test_data.columns)
print(test_data_scaled.head())

   aantal_kamers  vraagprijs
0       0.660483    0.199449
1      -0.053553   -0.997164
2       1.374518    0.668867
3       1.374518   -1.624033
4      -0.767588   -0.102483


In [40]:
test_data_scaled.std()

aantal_kamers    1.0005
vraagprijs       1.0005
dtype: float64

Tot slot is er een belangrijk en veelgebruikt optimalisatie algoritme genaamd Gradient Descent. Dit algoritme gaat iteratief op zoek naar de beste combinatie van parameters die de error minimaliseren (het verschil tussen de voorspelling en de werkelijkheid). Wanneer je de data niet schaalt kan het *heel* erg lang duren voordat het algoritme een optimale oplossing heeft gevonden aangezien het algoritme moet 'omlopen':

<img src="https://cdn-images-1.medium.com/max/1600/1*vXpodxSx-nslMSpOELhovg.png">

Let op dat er meerdere scaling / normalisatie technieken zijn en afhankelijk van jouw dataset wil je mogelijk meerdere technieken proberen. Meer informatie is te vinden op <a href="https://becominghuman.ai/demystifying-feature-scaling-baff53e9b3fd">demystifying-feature-scaling</a>. De StandardScaler() zit op het moment van schrijven standaard in de train_basic_models() functie en wordt op ieder model toegepast. 

## Stap 4: feature selection

In de modelbackbone uit DWH zit een zeer groot aantal variabelen en je kunt ervan uitgaan ze niet allemaal evenveel van belang zijn voor hetgeen je wil modelleren. Om te bepalen welke variabelen wel veel invloed hebben en je sowieso mee wil nemen in je uiteindelijke model, draai je in deze stap een aantal basismodellen om de feature importance te bepalen. Op die manier kun je na deze stap de variabelen die niets doen voor je afhankelijke variabele verwijderen. Dat bespaart een hoop computation costs tijdens het runnen van het model en zorg er bovendien voor dat je uitkomsten overzichtelijk blijven en makkelijker te presenteren.

Bovendien bestaan er een groot aantal classification algoritmes, met elk eigen voordelen en nadelen. Bij verschillende datasets kunnen zij verschillende prestaties vertonen. Om die reden kan het slim zijn enkele verschillende modellen te testen om te weten te komen welk model geschikt is voor jouw dataset. Met dat inzicht kun je het best presterende model gaan optimaliseren. Hierbij kan het verstandig zijn een afweging te maken tussen enerzijds de accuracy score van het model en anderzijds te running time van een model. Als een model dat veel langer runt maar minimaal beter presteert, kies je wellicht liever voor het snellere model. Ook kun je in je keuze laten meewegen hoe moeilijk of makkelijk de resultaten van het model te presenteren zijn. 

Tevens moet het model weten wat je x-variabelen zijn en wat je y-variabele is. Gebruik daarvoor onderstaand blok.

In [ ]:
y = train_set['y_var']
X = train_set.drop('y_var', axis=1)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=1337)

Nu is het tijd om een aantal verschillende basale classification modellen te trainen om de feature importance te bepalen en een idee te krijgen van de potentie van een algoritme. Om dit te kunnen doen definieer je eerst de y- en de x-variabelen(n). Dat kan simpel met onderstaande code wanneer je de y-variabele y_var hebt genoemd.

Vervolgens gaan we de train set verder opsplitsen, maar let op: de holdout set blijft onaangeraakt. De train set wordt opgesplitst in een train en een validation set. De train set zal door de modellen gebruikt worden om de algoritmes te fitten, terwijl de validation set zal worden gebruikt om te bepalen hoe goed de fit is. Ook nu is het mogelijk om de verdeling van de sets aan te passen en vast te houden hoe de random verdeling plaatsvindt.

Daarna worden vier classification models getraind om de feature importance te bepalen. Het doel van van het draaien van de modellen in deze fase is nog niet dat de modellen optimaal werken, wel dat ze snel een overzicht geeft van de feature importance en de potentie van het modellen. We behandelen de volgende modellen:
- Tree based classifiers
    - Random forest
    - XGBClassifier
    - ExtraTreesClassificer
    - SGDClassifier
- Linear classifiers
    - Logistic regression
    - LightGBMClassifier

Hier beneden worden deze technieken één voor één uitgelegd.

### Random forest

Random forest bundelt de resultaten van meerdere decision trees die simultaan worden gerund met verschillende random subsets (met terugleggen) van features en verschillende random subsamples van de trainingset. 
<img src="https://www.amo-nl.com/wp-content/uploads/2019/08/Random-Forest.png">
Via onderstaande links vind je meer documentatie over dit algorithme:
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
- https://www.datacamp.com/community/tutorials/random-forests-classifier-python
- https://www.amo-nl.com/het-principe-van-de-werking-van-random-forest/

Onderstaand blok code resulteert in een aanbeveling aangaande de features om mee te nemen in het finale model.

In [ ]:
print(n_features_rf)

### XGBoostClassifier 

Extreme Gradient Boosting maakt net als random forest gebruik van meerdere decision trees, maar in tegenstelling tot random forest wordt iedere tree op de volledige training set gefit in plaats van op subsamples.Bovendien gebeurt het runnen van de trees achter sequentieel in plaats van parallel. Hierbij heeft iedere nieuwe tree als doel heeft de voorgaande tree te verbeteren met een extra stukje informatie. Via onderstaande links vind je meer documentatie over dit algoritme:
- https://xgboost.readthedocs.io/en/latest/python/python_intro.html
- https://xgboost.readthedocs.io/en/latest/get_started.html
- https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/
- https://statquest.org/2019/12/16/xgboost-part-1-xgboost-trees-for-regression/
- https://statquest.org/2020/01/18/xgboost-part-2-xgboost-trees-for-classification/

Het runnen van de code hieronder resulteert in een lijst van features die geselecteerd worden om mee te nemen in het model volgens de XGBoostClassifier.

### ExtraTreesClassificer

ExtraTrees maakt wederom gebruikt van individuele decision trees, maar dit algoritme gebruikt subsamples zonder terugleggen en gebruikt bij iedere vertakking een random cutoff point voor de feature in plaats van een geoptimaliseerd cutoff point. Via onderstaande links vind je meer documentatie over dit algoritme:
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
- https://towardsdatascience.com/an-intuitive-explanation-of-random-forest-and-extra-trees-classifiers-8507ac21d54b
- https://www.youtube.com/watch?v=Q1qpG7gwix4

### Logistic regression

Logistic regression is een variant die lijkt op een lineare regressie, met als belangrijkste verschil dat logistic regression een binaire uitkomst schat in plaats van een continue uitkomst. Lees hier meer over logistische regressie:

- https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
- https://statquest.org/2018/03/05/statquest-logistic-regression/

### SGDClassifier
SGDClassifier is een algoritme dat een linear regressie toepast met minimalisering van de loss functie. Dat doet het algoritme door voor random samples de loss bij toepassing van het regressie te bepalen en deze kennis mee te nemen bij het toepassen van een regressie op een nieuwe sample.
Stochastic gradient descent (SGD) learning: the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate).

What is SGD Classifier?
SGD Classifier implements regularised linear models with Stochastic Gradient Descent.
So, what is stochastic gradient descent?
Stochastic gradient descent considers only 1 random point while changing weights unlike gradient descent which considers the whole training data. As such stochastic gradient descent is much faster than gradient descent when dealing with large data sets. Here is a nice answer on Quora which explains in detail the difference between gradient descent and stochastic gradient descent.

### LightGBMClassifier

LightGBM is tevens een gradient boosting classifier dat gebruikt maken van tree based learning algorithms, maar werkt leaf-wise in plaats van level-wise en is daarmee efficiënter in snelheid en geheugen dan andere algoritmes.
<img src="https://miro.medium.com/max/1211/1*AZsSoXb8lc5N6mnhqX5JCg.png">
<img src="https://miro.medium.com/max/983/1*whSa8rY4sgFQj1rEcWr8Ag.png">
Meer uitleg over LightGBM vind je hier:
- https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc
- https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf

**Hier nu duidelijk handvaten geven over de beslissingen die nu moeten worden genomen (welk model en welke features) en gabseerd waarop!**

**Als je bovenstaande stappen hebt doorlopen, heb je nu vier lists van features die volgens een model worden gedefinieerd als informatie voor het inschatten van de Y-variabele. Het kan zijn dat die vier lists veelal dezelfde features bevatten, maar flinke afwijkingen in het aantal features of de inhoud van de features kan ook voorkomen. Het is verstanding om bij de volgende stap business kennis mee te nemen bij het maken van de beslissing welke features verder mee worden genomen in het model. Er zou kunnen worden gekozen om alle features die hierboven als informatief zijn bestempeld door minimaal één van de modellen mee te nemen. Dat kun je doen met behulp van onderstaand stukje code, waarbij de lists worden samengevoegd en de duplicaten worden verwijderd.**

**Afhankelijk van de wensen van het model, zijn andere methoden ook mogelijk. Bijvoorbeeld: wanneer je een classification model bouwt om meer inzicht te krijgen in drivers, kan het wenselijk zijn een kleiner aantal features te pakken. Dan kun je ervoor kiezen om alleen de features mee te nemen die bij alle vier de modellen naar boven kwamen.**

## Stap 6: run models, determine best model, selected model met hyperparameters

Als het goed is, heb je nu beter begrip van een aantal veelgebruikte voorspelmodellen. Nu is het tijd om de resultaten van deze modellen te gaan vergelijken. Op die manier kun je makkelijker beoordelen welk model het meest geschikt is voor jouw vraagstuk. Maar let op: bij deze afweging komt meer kijken dan alleen modelresultaten. Hier is het vooral van belang dat je ook praktische overwegingen en business kennis meeneemt!

Met onderstaand blok fit je de hierboven genoemde modellen voor jouw dataset en print je hoe ze presteren op het gebied van: 
- de tijd die het kost om het model te fitten
- **param_classifier?**
- **sum_rank?**
- accuracy
- precision
- **mean_test?**
- ROC AUC


### Stap 5a: train model

In [ ]:
train_basic_models(x_test, y_test)

### Stap 5b: predict model

### Stap 5c: determine model diagnostics

In [ ]:
plot_recall_precision(model, x_val, y_val)

In [ ]:
undefit_overfit(model, x_train, y_train, x_val, y_val)

Model evaluation metrics: https://neptune.ai/blog/evaluation-metrics-binary-classification